In [1]:
% matplotlib inline

import numpy as np  # import numpy library as np
import math         # use mathematical functions defined by the C standard
import matplotlib.pyplot as plt # import pyplot library as plt 
import pandas as pd # import pandas library as pd

from numpy import fft
from datetime import datetime

plt.style.use('ggplot') # use "ggplot" style for graphs

pltparams = {'legend.fontsize':16,'axes.labelsize':20,'axes.titlesize':20,
             'xtick.labelsize':12,'ytick.labelsize':12,'figure.figsize':(7.5,7.5),}
plt.rcParams.update(pltparams)

In [14]:
r = .6
N = 4
ps = np.array([True, True, True, True])
pswitch  = np.array([r, 1.0-r])
switch = np.random.choice([True,False], p=pswitch,size=N)
ps[switch] = False

print(switch)
print(ps)

[ True  True  True  True]
[False False False False]


In [15]:
# Original two dealer code

#params={'L':0.01,'c':0.01,'dp':0.01,'dt':0.01**2,'d':1.00,'M':10} # define model parameters
#p0 = np.ones(2)*100.25

#params={'N':100,'r':0.6,'L':0.01,'c':0.01,'dp':0.01,'dt':0.01**2, 'd':1.00, 'M':10} # define model parameters
#p0 = np.ones(params['N'])*100.25

params={'e0':1e-4,'sigma':0.1,'N':100,'r':0.6,'L':0.01,'c':0.01,'dp':0.01,'dt':0.01**2, 'd':1.00, 'M':10} # define model parameters
p0  = np.ones(params['N'])*100.25


def model2(params,p0,numt):
    ## switching rates
    pswitch  = np.array([params['r'], 1.0-params['r']]) 
    
    # compute running average Eq.(L6)
    def avgprice(dpn): 
        M = len(dpn)     
        weights  = np.array(range(1,M+1))*2.0/(M*(M+1))
        return weights.dot(dpn)

    ## return transaction candidate: pair (i,j) with largest price difference
    def transactionPair(pt):  
        return [np.min(pt), np.max(pt)]
    
    # update prices for transaction between the two dealers and return new market price
    #def transactionUpdate(pt):
    #    newprice = np.average(pt)  #set mid-prices to new market price Eq.(L2)
    #    pt[:]    = newprice
    #   return newprice
    
    # update prices for transaction between the two dealers (i,j) and return new market price
    def transactionUpdate(i,j,pt):
        newprice = np.average([pt[i], pt[j]])
        switch   = np.random.choice([True,False], p=pswitch,size=params['N']) # determine which dealers update price
        switch[i]=switch[j]=True
        pt[switch] = newprice
        return newprice    
    
    # ... variable initializations ...
    mktprice = np.zeros(numt)   # initialize market price P(n)
    dmktprice= np.zeros(numt)   # initialize change in price dP(n) needed for running average
    ticktime = np.zeros(numt,dtype=np.int) #initialize array for tick times
    price    = p0               #initialize dealer's mid-price (p1,p2)
    time,tick= 0,0 # real time(t) and time time (n)
    deltapm  = 0.0 # trend term d <dP>_m dt for current random walk
    cdp      = params['c']*params['dp'] # define random step size
    ddt      = params['d']*params['dt'] # define amplitude of trend term
    
    deltapm  = 0.0                                           # trend term <dP>_m dt for current random walk
    dbar     = params['d']                                   # define average trend following effect \bar{d}
    di       = params['sigma']*np.random.randn(params['N'])  # trend following parameter for dealer i
    
    while tick < numt: # loop over ticks
        [i,j]    = transactionPair(price)   # transaction candidate
        #while np.abs(price[0]-price[1]) < params['L']: # transaction criterion Eq.(L1)
        while np.abs(price[i]-price[j]) < params['L']: # transaction criterion Eq.(L1)
            #price = price + deltapm + np.random.choice([-cdp,cdp], size=2) # random walk step Eq.(L4)
            price = price + deltapm + np.random.choice([-cdp,cdp], size=params['N']) # random walk step Eq.(L4)
            [i,j] = transactionPair(price) # update transaction candidate
            time += 1 #update real time 

        #mktprice[tick] = transactionUpdate(price)                      # save market price
        mktprice[tick] = transactionUpdate(i,j,price)
        
        # ... finalize loop ...
        dmktprice[tick]= mktprice[tick] - mktprice[np.max([0,tick-1])] # save dP(n) = P(n) - P(n-1)
        ticktime[tick] = time # save transaction time
        tick += 1             #update ticks
        tick0 = np.max([0, tick - params['M']])       #compute tick start for running average
        deltapm = avgprice(dmktprice[tick0:tick])*ddt #compute updated trend term for current tick time
    return ticktime,mktprice